In [ ]:
! pip install datasets transformers rouge-score nltk wandb sentencepiece 

     |████████████████████████████████| 245kB 27.7MB/s 
     |████████████████████████████████| 2.3MB 46.6MB/s 
     |████████████████████████████████| 1.8MB 47.8MB/s 
     |████████████████████████████████| 1.2MB 56.7MB/s 
     |████████████████████████████████| 122kB 56.2MB/s 
     |████████████████████████████████| 245kB 39.0MB/s 
     |████████████████████████████████| 3.3MB 48.3MB/s 
     |████████████████████████████████| 901kB 47.6MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 133kB 56.7MB/s 
     |████████████████████████████████| 174kB 57.3MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=5974da35c2d0629dc3796ed1601defe2c938eecf1c3f1a9d5b4bc49ae456766c
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37

In [ ]:
#!git clone https://github.com/huggingface/transformers
#%cd transformers
#!pip install .


#!pip install -r ./transformers/examples/pytorch/summarization/requirements.txt


# Install env

### Install triton

In [ ]:
!apt-get install llvm-9-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binfmt-support libffi-dev libllvm9 libpfm4 llvm-9 llvm-9-runtime
  llvm-9-tools python-chardet python-pkg-resources python-pygments python-yaml
  python3-pkg-resources python3-pygments python3-yaml
Suggested packages:
  llvm-9-doc python-setuptools ttf-bitstream-vera python3-setuptools
The following NEW packages will be installed:
  binfmt-support libffi-dev libllvm9 libpfm4 llvm-9 llvm-9-dev llvm-9-runtime
  llvm-9-tools python-chardet python-pkg-resources python-pygments python-yaml
  python3-pkg-resources python3-pygments python3-yaml
0 upgraded, 15 newly installed, 0 to remove and 39 not upgraded.
Need to get 48.8 MB of archives.
After this operation, 287 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-yaml amd64 3.12-1build2 [109 kB]
Get:2 http://archive.ubuntu.com/ubuntu bi

In [ ]:
!pip install cpufeature

In [ ]:
!pip install triton==0.2.3

     |████████████████████████████████| 952kB 34.3MB/s 


### Install DeepSpeed

In [ ]:
!pip install libaio

In [ ]:
!rm /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
! apt install libaio-dev
!DS_BUILD_TRANSFORMER=1 DS_BUILD_STOCHASTIC_TRANSFORMER=1 DS_BUILD_UTILS=1 
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install transformers[deepspeed]

#### Test installation: we should have the following output

In [ ]:
!ds_report

### Download repo and install other libs

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

In [ ]:
!pip install natsort

# Play


In [ ]:
!CUDA_LAUNCH_BLOCKING=1
import logging

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:
student_number = 4
data_name = 'xsum'
#model_name = f'google/pegasus-{data_name}'
model_name = 'tuner007/pegasus_paraphrase'

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

#directory = f'/content/drive/MyDrive/GP/dataset'
#if not os.path.exists(directory):
#    os.makedirs(directory)

PATH = f'/content/drive/MyDrive/GP/student_{data_name}_{student_number}'


In [ ]:
from datasets import load_dataset, load_metric, load_from_disk

#tokenized_datasets = load_from_disk(f'{PATH}/../dataset/{data_name}')
tokenized_datasets = load_from_disk('/content/drive/MyDrive/GP/dataset/xsum')
#raw_datasets = load_dataset(data_name)
metric = load_metric("rouge")

In [ ]:
tokens = tokenized_datasets.filter(lambda example, indice: indice < 1001, with_indices=True)

In [ ]:
tokenized_datasets

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Model Preparation

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer

max_input_length = 1024
max_target_length = 60

teacher = AutoModelForSeq2SeqLM.from_pretrained(model_name, max_length = max_input_length, max_position_embeddings=max_input_length)


In [ ]:
import datasets
import warnings
import torch
from torch import nn
from typing import Optional, Tuple, List, Union
from transformers import PegasusModel, PegasusConfig, PegasusForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedModel
from transformers import SummarizationPipeline
students_config_book = {
    '2': PegasusConfig(encoder_layers=2, decoder_layers=2),
    '4': PegasusConfig(encoder_layers=4, decoder_layers=4),
    '6': PegasusConfig(encoder_layers=6, decoder_layers=6),
    '8': PegasusConfig(encoder_layers=8, decoder_layers=8),
    '10': PegasusConfig(encoder_layers=10, decoder_layers=10),
    '12': PegasusConfig(encoder_layers=12, decoder_layers=12),
    '16': PegasusConfig(encoder_layers=16, decoder_layers=16)
}


LAYERS_TO_COPY = {
    16: {  # maps  num layers in student -> which teacher layers to copy
        1: [0],
        2: [0, 15],
        3: [0, 8, 15],
        4: [0, 5, 10, 15],
        6: [0, 3, 6, 9, 12, 15],
        8: [0, 2, 4, 6, 8, 10, 12, 15],
        9: [0, 1, 3, 5, 7, 9, 11, 13, 15],
        12: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15],
        16: list(range(16)),
    },
}
LAYERS_TO_SUPERVISE = {
    # maps  num layers in student -> which teacher layers to copy.
    16: {1: [15], 4: [4, 9, 12, 15], 8: [1, 3, 5, 7, 9, 11, 13, 15]},
}


def copy_layers(src_layers: nn.ModuleList, dest_layers: nn.ModuleList, layers_to_copy) -> None:
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(
        layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

# Copied from transformers.models.bart.modeling_bart.shift_tokens_right


def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    assert pad_token_id is not None, "self.model.config.pad_token_id has to be defined."
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids


def pick_layers_to_copy(n_student, n_teacher):
    try:
        val = LAYERS_TO_COPY[n_teacher][n_student]
        return val
    except KeyError:
        if n_student != n_teacher:
            warnings.warn(
                f"no hardcoded layers to copy for teacher {n_teacher} -> student {n_student}, defaulting to first {n_student}"
            )
        return list(range(n_student))


def get_layers_to_supervise(n_student, n_teacher) -> List[int]:
    """Used or the --supervise_forward kwarg"""
    if n_student > n_teacher:
        raise ValueError(
            f"Cannot perform intermediate supervision for student {n_student} > teacher {n_teacher}")
    elif n_teacher == n_student:
        return list(range(n_teacher))
    elif n_student == 1:
        return [n_teacher - 1]
    else:
        return LAYERS_TO_SUPERVISE[n_teacher][n_student]


def create_student_with_configuration(teacher,
                                      e=None,
                                      d=None,
                                      copy_first_teacher_layers = False,
                                      save_path='./student'):

    teacher.eval()
    teacher_e, teacher_d = teacher.config.encoder_layers, teacher.config.decoder_layers
    init_kwargs = teacher.config.to_diff_dict()
    if e is None:
        e = teacher_e
    if d is None:
        d = teacher_d
    init_kwargs.update({"encoder_layers": e, "decoder_layers": d})
    student_cfg = teacher.config_class(**init_kwargs)
    student = AutoModelForSeq2SeqLM.from_config(student_cfg)
    # Start by copying the full teacher state dict this will copy the first N teacher layers to the student.
    info = student.load_state_dict(teacher.state_dict(), strict=False)
    # every student key should have a teacher keys.
    assert info.missing_keys == [], info.missing_keys

    if copy_first_teacher_layers:  # Our copying is done. We just log and save
        e_layers_to_copy, d_layers_to_copy = list(range(e)), list(range(d))
        student.save_pretrained(save_path)
        return student, e_layers_to_copy, d_layers_to_copy

    # Decide which layers of the teacher to copy. Not exactly alternating -- we try to keep first and last layer.
    e_layers_to_copy: List[int] = pick_layers_to_copy(e, teacher_e)
    d_layers_to_copy: List[int] = pick_layers_to_copy(d, teacher_d)

    copy_layers(teacher.model.encoder.layers,
                student.model.encoder.layers, e_layers_to_copy)
    copy_layers(teacher.model.decoder.layers,
                student.model.decoder.layers, d_layers_to_copy)

    student.config.init_metadata = dict(
        teacher_type=teacher.config.model_type,
        copied_encoder_layers=e_layers_to_copy,
        copied_decoder_layers=d_layers_to_copy,
    )
    #student.save_pretrained(save_path)
    # Save information about copying for easier reproducibility

    return student


In [ ]:
student = create_student_with_configuration(
                                      teacher,
                                      e=16,
                                      d=student_number,
                                      copy_first_teacher_layers = False,
                                      save_path=PATH).to('cuda')

del teacher
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
for param in student.model.shared.parameters():
  param.requires_grad = False
for param in student.model.encoder.parameters():
  param.requires_grad = False

In [ ]:
#before
student.model.decoder.layers[0].final_layer_norm.weight

In [ ]:
#after
student.model.decoder.layers[0].final_layer_norm.weight

# The Training Loop

In [ ]:
import nltk
nltk.download('punkt')
 
import numpy as np
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:

class MyTrainer(Seq2SeqTrainer):
    
    def shift_tokens_right(self,input_ids: torch.Tensor, pad_token_id = 0, decoder_start_token_id = 0):
      """
      Shift input ids one token to the right.
      """
      shifted_input_ids = input_ids.new_zeros(input_ids.shape)
      shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
      shifted_input_ids[:, 0] = decoder_start_token_id

      assert pad_token_id is not None, "self.model.config.pad_token_id has to be defined."
      # replace possible -100 values in labels by `pad_token_id`
      shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

      return shifted_input_ids
    
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"]
        pad_token_id = tokenizer.pad_token_id
        decoder_input_ids = self.shift_tokens_right(labels, pad_token_id)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        #print(f"input: {input_ids}, masks: {attention_mask}")
        outputs = model(**inputs)
        logits = outputs["logits"]
        #print(f'logits: {logits}, labels: {decoder_input_ids}')
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, logits.shape[-1]),labels.view(-1))
        #print(loss)
        return (loss, outputs) if return_outputs else loss

In [ ]:
%%bash
cat <<"EOT" > ds_config_zero2.json
{
    "fp16":{
        "enabled":false,
        "hysteresis":2,
        "initial_scale_power":32,
        "loss_scale":0,
        "loss_scale_window":1000,
        "min_loss_scale":1
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "amp": {
      "enabled":false,
      "opt_level": "O1"
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },

    "zero_optimization": {
      "stage": 2,
      "offload_optimizer": {
          "device": "cpu",
          "pin_memory": true
      },
      "allgather_partitions": true,
      "allgather_bucket_size": 2e8,
      "reduce_scatter": true,
      "reduce_bucket_size": 2e8,
      "overlap_comm": true,
      "contiguous_gradients": true
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 500,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
EOT


In [ ]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994' # modify if RuntimeError: Address already in use
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir=f'{PATH}/output',
    evaluation_strategy = "steps",
    save_strategy = 'epoch',
    eval_steps = 500,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size-8,
    gradient_accumulation_steps = 16,
    eval_accumulation_steps = 16,
    warmup_steps=500,
    weight_decay=0.01,
    #weight_decay=0,
    logging_dir=f'{PATH}/logs',            # directory for storing logs
    logging_steps=10,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    load_best_model_at_end = True,
    #adafactor = True,
    #fp16=True,
    deepspeed = "ds_config_zero2.json"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=student)


In [ ]:
from transformers import Adafactor, EarlyStoppingCallback
#optimizer = Adafactor(student.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)

trainer = MyTrainer(
    student,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokens["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #optimizers= (optimizer,_)
    callbacks = [EarlyStoppingCallback]
)

In [ ]:
trainer.train('/content/drive/MyDrive/GP/student_xsum_4/output/checkpoint-2000')

[2021-06-11 15:40:00,678] [INFO] [logging.py:60:log_dist] [Rank 0] DeepSpeed info: version=0.4.0, git-hash=unknown, git-branch=unknown
[2021-06-11 15:40:00,693] [INFO] [utils.py:13:_initialize_parameter_parallel_groups] data_parallel_size: 1, parameter_parallel_size: 1


06/11/2021 15:40:00 - INFO - root -   Added key: store_based_barrier_key:2 to store for rank: 0


[2021-06-11 15:40:00,925] [INFO] [engine.py:173:__init__] DeepSpeed Flops Profiler Enabled: False
[2021-06-11 15:40:01,443] [INFO] [engine.py:693:_configure_optimizer] Using DeepSpeed Optimizer param name adamw as basic optimizer
[2021-06-11 15:40:01,445] [INFO] [engine.py:697:_configure_optimizer] DeepSpeed Basic Optimizer = DeepSpeedCPUAdam
Checking ZeRO support for optimizer=DeepSpeedCPUAdam type=<class 'deepspeed.ops.adam.cpu_adam.DeepSpeedCPUAdam'>
[2021-06-11 15:40:01,453] [INFO] [logging.py:60:log_dist] [Rank 0] Creating fp16 ZeRO stage 2 optimizer
[2021-06-11 15:40:01,460] [INFO] [stage2.py:105:__init__] Reduce bucket size 200000000.0
[2021-06-11 15:40:01,461] [INFO] [stage2.py:106:__init__] Allgather bucket size 200000000.0
[2021-06-11 15:40:01,462] [INFO] [stage2.py:107:__init__] CPU Offload: True
Using /root/.cache/torch_extensions as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/utils...
Emitting ninja build file /root/.cache/torch_ex

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

[2021-06-11 15:49:39,540] [INFO] [timer.py:160:stop] 0/500, SamplesPerSec=18.582311372347352
[2021-06-11 15:56:57,337] [INFO] [timer.py:160:stop] 0/1000, SamplesPerSec=18.516574226680017
[2021-06-11 16:04:15,055] [INFO] [timer.py:160:stop] 0/1500, SamplesPerSec=18.49506367651793
[2021-06-11 16:11:32,473] [INFO] [timer.py:160:stop] 0/2000, SamplesPerSec=18.487094198768858
[2021-06-11 16:18:49,537] [INFO] [timer.py:160:stop] 0/2500, SamplesPerSec=18.48510646609173
[2021-06-11 16:26:06,847] [INFO] [timer.py:160:stop] 0/3000, SamplesPerSec=18.48220620850975
[2021-06-11 16:33:24,063] [INFO] [timer.py:160:stop] 0/3500, SamplesPerSec=18.48042918875149
[2021-06-11 16:40:41,188] [INFO] [timer.py:160:stop] 0/4000, SamplesPerSec=18.479593321275228
[2021-06-11 16:47:58,237] [INFO] [timer.py:160:stop] 0/4500, SamplesPerSec=18.479709925598268
[2021-06-11 16:51:38,542] [INFO] [logging.py:60:log_dist] [Rank 0] step=1500, skipped=0, lr=[1e-05], mom=[[0.9, 0.999]]
[2021-06-11 16:55:15,144] [INFO] [timer

06/11/2021 17:45:30 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


[2021-06-11 17:45:41,310] [INFO] [logging.py:60:log_dist] [Rank 0] Saving model checkpoint: /content/drive/MyDrive/GP/student_xsum_4/output/checkpoint-2500/global_step1703/mp_rank_00_model_states.pt
[2021-06-11 17:46:36,141] [INFO] [engine.py:1938:_copy_recovery_script] creating recovery script /content/drive/MyDrive/GP/student_xsum_4/output/checkpoint-2500/zero_to_fp32.py
[2021-06-11 17:46:36,162] [INFO] [engine.py:1951:_save_zero_checkpoint] zero checkpoint saved /content/drive/MyDrive/GP/student_xsum_4/output/checkpoint-2500/global_step1703/zero_pp_rank_0_mp_rank_00_optim_states.pt
[2021-06-11 17:53:59,963] [INFO] [timer.py:160:stop] 0/8500, SamplesPerSec=18.479279730714435
[2021-06-11 18:01:17,262] [INFO] [timer.py:160:stop] 0/9000, SamplesPerSec=18.479194840341012


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
2500,6.131500,5.691840,29.645300,9.929800,23.829100,23.819100,21.812200


[2021-06-11 18:08:34,485] [INFO] [timer.py:160:stop] 0/9500, SamplesPerSec=18.478683226674317
[2021-06-11 18:15:51,886] [INFO] [timer.py:160:stop] 0/10000, SamplesPerSec=18.477736050236395
[2021-06-11 18:23:09,025] [INFO] [timer.py:160:stop] 0/10500, SamplesPerSec=18.4774809733934
[2021-06-11 18:30:26,170] [INFO] [timer.py:160:stop] 0/11000, SamplesPerSec=18.477148935121654
[2021-06-11 18:37:43,127] [INFO] [timer.py:160:stop] 0/11500, SamplesPerSec=18.477105803654272
[2021-06-11 18:45:00,211] [INFO] [timer.py:160:stop] 0/12000, SamplesPerSec=18.476852695298984
[2021-06-11 18:52:16,710] [INFO] [timer.py:160:stop] 0/12500, SamplesPerSec=18.477619840712027
[2021-06-11 18:55:56,794] [INFO] [logging.py:60:log_dist] [Rank 0] step=2000, skipped=0, lr=[1e-05], mom=[[0.9, 0.999]]
[2021-06-11 18:59:33,220] [INFO] [timer.py:160:stop] 0/13000, SamplesPerSec=18.478262398721707
[2021-06-11 19:06:50,026] [INFO] [timer.py:160:stop] 0/13500, SamplesPerSec=18.47837813641359
[2021-06-11 19:14:07,282] [IN

In [ ]:
#trainer.save_model(PATH + '/output/')

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(tokens["test"])

In [ ]:
import wandb 
wandb.init()